# Introduction

The aim of this notebook is to calculate an average velocity vector field for each video in an experiment. The notebook uses the registration process to do this:

1. Each frame in a video is split up into regions of size `REGION_SIZE` (e.g. (9,9) ). Within the centre of each region a pattern is isolated of size `PTRN_SIZE` (e.g. (5,5) ).
    - At time step *T*, the pattern in a region is isolated. 
    - This pattern is then "registered" with another pattern in the same region but at time step *T+1*.
    - If a suitable match for the pattern is found, then a vector for the motion is calculated, otherwise a zero-vector is returned.
    - Note that the resulting velocity vector field will have a different size to that of the original video. For example, a (2048,2048) frame will result in a (408, 408) velocity vector field.
    
    <img src="markdown_images_for_notebooks/registration-example.PNG">

2. The `MAX_WINDOW` parameter reduces the number of frames in a video. (e.g. `MAX_WINDOW = 5`)
    - A maxpool operation is applied every (for example) 5 frames so if the video consists of 300 frames, this is reduce to 60 frames.
    - This is done to increase the signal in each aggregate frame to make registration easier 
    - Signal may need to be increased in cases where beads submerge below the liquid surfaces between frames and then re-emerge
    
    
3. The errors associated with the registration process have also been identified and studied further in this [Notebook: 1_errors_in_registration](auxiliary_analysis/1_errors_in_registration.ipynb)

    - The registration process in the current notebook is carried out for both the forwards and backwards time directions
    - In theory, if the process is successful the equal but opposite vector field for the backwards time direction video should be returned. 
    - The validation of the registration process can be found in this [Notebook: 3_validate_registration](auxiliary_analysis/3_validate_registration.ipynb)
    

4. In total, there are 3 adjustable parameters that can be tuned: `REGION_SIZE`, `PTRN_SIZE`, `MAX_WINDOW`
    
    - These parameters can vary from one experiment to another. As mentioned above, the combination of `REGION_SIZE` and `PTRN_SIZE` sets a limit on the maximum velocity vector that can be calculated
    - `MAX_WINDOW` can also be varied - there maybe some experiments where beads tend not to sub merge below the liquid surface and so maxpooling across frames may not be required
    - Additionally, if the beads are slow moving in an experiment, then rather than maxpooling across frames, it may be more beneficial to simply skip frames.
    - The tuning of these parameters is carried out in this [Notebook: 2_minimise_error_in_registration](auxiliary_analysis/2_minimise_error_in_registration.ipynb). This notebook must be run for each experiment separately.

# Imports

In [ ]:
import numpy as np
import scipy as sp
from scipy import signal
import skimage
from skimage import morphology as sk_morph
from skimage import exposure
import os
import json

from IPython import display
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from tqdm import tqdm

from fam13a import utils, image, register
from multiprocessing import Pool
from functools import partial

# Constants

In [ ]:
PROJ_ROOT = utils.here()
# declare the data input directory
INTERIM_HBEC_ROOT = os.path.join(PROJ_ROOT, 'data', 'interim', 'hbec')
print(os.listdir(INTERIM_HBEC_ROOT))

In [ ]:
# choose experiment data to load
EXP_ID = 'N67030-59_8_perc'

INTERIM_ROOT = os.path.join(PROJ_ROOT, INTERIM_HBEC_ROOT, EXP_ID)

# declare the various output directories
PROCESSED_ROOT = os.path.join(PROJ_ROOT, 'data', 'processed', 'hbec', EXP_ID)
ROI_ROOT = os.path.join(PROCESSED_ROOT, 'roi')
SEG_ROOT = os.path.join(PROCESSED_ROOT, 'segmented', 'movement')
MAX_FRAME_ROOT = os.path.join(PROCESSED_ROOT, 'max_frame')

OUTPUT_ROOT = os.path.join(PROCESSED_ROOT, 'register')
# set level of parallelisation
NCPUS = 32
# define the common file extension used in the input data files
EXTENSION = '.ome.tif'

# find all relevant data files in the data directory 
files = sorted([_f for _f in os.listdir(INTERIM_ROOT) if _f.endswith('tif')])
# remove the extension the file names, se we keep only the bit with useful information
files = [_f.split(EXTENSION)[0] for _f in files]
print(files)

# Setup

Set registration parameters

In [ ]:
# window size to take along the time dimension
MAX_WINDOW = 5
# set the sizes of the pattern and search regions
PTRN_SIZE = (5,)*2
REGION_SIZE = (9,)*2

defining a very simple wrapper around the register.estimate_shifts functon purely to allow the inclusion of a progress bar during processing

# Process

- For each video, calculate the shifts in both the forwards and backwards time directions
- NOTE: it takes ~60 seconds to process 1 pair of frames on 1 CPU
- with NCPUS=32, time is 65 min (for a 24 well experiment)

In [ ]:
for idx, _file in enumerate(files):
    print(f'starting: {_file} ({idx+1} of {len(files)})')
    
    # load the frames and create forwards and backwards in time copies
    frames = utils.frames_from_stack(os.path.join(INTERIM_ROOT, f'{_file}{EXTENSION}'))
    frames_forwards = frames.copy()
    frames_backwards = frames.copy()[::-1]
    # load each of the pther associated interim/processed files
    roi = np.load(os.path.join(ROI_ROOT, f'{_file}.npy'))
    max_frame = np.load(os.path.join(MAX_FRAME_ROOT, f'{_file}.npy'))
    movement_mask = np.load(os.path.join(SEG_ROOT, f'{_file}.npy')).astype(bool)
    
    # need to run the reg process forwards and backwards in time - only need the 
    # backwards shifts since the unpad-slice and sub-shape variables are independent
    # of time direction
    reg_process_forwards = register.run_registration_process(
        frames_forwards, MAX_WINDOW, PTRN_SIZE, REGION_SIZE, NCPUS
    )
    shifts_forwards = reg_process_forwards['shifts']
    reg_process_backwards = register.run_registration_process(
        frames_backwards, MAX_WINDOW, PTRN_SIZE, REGION_SIZE, NCPUS
    )
    shifts_backwards = reg_process_backwards['shifts']
    
    # given the forwards and backwards shifts, we can caluclate the average velocity field
    # then we can calculate the angles between the vectors in the positive forwards 
    # velocity field and (negative) backwards velocity field.
    # So a small angle would correspond to a succesful registration process and a large
    # angle above some threshold would correspond to an unsuccesful registration process.
    velocity_forwards = register.calculate_mean_velocity_field(shifts_forwards)['normalised_velocity']
    velocity_backwards = register.calculate_mean_velocity_field(shifts_backwards)['normalised_velocity']
    validation_angles = register.calculate_angles_for_validation(velocity_forwards, velocity_backwards)
    
    # get the unpad_slice and sub_shape arrays from forwards pass (these
    # arrays are independent of time direction)
    unpad_slice = reg_process_forwards['unpad_slice']
    sub_shape = reg_process_forwards['sub_shape']
    
    # construct the downsampled movement mask
    sub_mask = image.patch.extract(movement_mask[unpad_slice, unpad_slice], PTRN_SIZE).max(axis=(-1, -2))
    sub_mask = sub_mask.reshape(sub_shape, sub_shape).astype(bool)

    # construct the downsampled max_frame projection
    # this projection is used for visualization purposes only
    sub_max_frame = frames.max(axis=0)[unpad_slice, unpad_slice]
    sub_max_frame = image.patch.extract(sub_max_frame, PTRN_SIZE).max(axis=(-1, -2))
    sub_max_frame = sub_max_frame.reshape(sub_shape, sub_shape)
    
    # construct the downsampled roi mask
    sub_roi = image.patch.extract(roi[unpad_slice, unpad_slice], PTRN_SIZE).max(axis=(-1, -2))
    sub_roi = sub_roi.reshape(sub_shape, sub_shape).astype(bool)
    
    # ensure the output directory exists
    register_dir = os.path.join(OUTPUT_ROOT, _file)
    os.makedirs(register_dir, exist_ok=True)
    
    # save the calculate frame shifts and downsampled versions of the movement mask and max_frame
    np.save(os.path.join(register_dir, 'shifts.npy'), shifts_forwards)
    np.save(os.path.join(register_dir, 'reverse_shifts.npy'), shifts_backwards)
    np.save(os.path.join(register_dir, 'mask.npy'), sub_mask)    
    np.save(os.path.join(register_dir, 'roi.npy'), sub_roi)
    np.save(os.path.join(register_dir, 'max_frame.npy'), sub_max_frame)
    np.save(os.path.join(register_dir, 'validation_angles.npy'), validation_angles)

    # save the registration parameters in the same folder
    params = {
        'max_window': MAX_WINDOW,
        'ptrn_size': PTRN_SIZE,
        'region_size': REGION_SIZE
    }
    with open(os.path.join(register_dir, 'params.json'), 'w') as json_f:
        json.dump(params, json_f)